# Description

Use incremental solving to compute the optimal solution and store agent-location-table in the original_plan.lp file

In [1]:
!python inc_orig.py encoding_orig.lp instances/env_17_multi_agent.lp --imax 100

inc-example version 1.0
Reading from encoding_orig.lp ...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Answer: 1
orig(agent(0),(25,5),0,(25,5)) orig(agent(1),(25,6),0,(25,6)) orig(agent(1),(25,6),1,(25,6)) orig(agent(0),(25,5),1,(25,5)) orig(agent(0),(25,5),2,(25,4)) orig(agent(1),(25,6),2,(25,6)) orig(agent(1),(25,6),3,(25,5)) orig(agent(0),(25,4),3,(25,3)) orig(agent(0),(25,3),4,(25,2)) orig(agent(1),(25,5),4,(25,4)) orig(agent(1),(25,4),5,(25,3)) orig(agent(0),(25,2),5,(25,1)) orig(agent(0),(25,1),6,(24,1)) orig(agent(1),(25,3),6,(25,2)) orig(agent(1),(25,2),7,(25,1)) orig(agent(0),(24,1),7,(23,1)) orig(agent(0),(23,1),8,(22,1)

Compute conflict locations for agents (Locations which are passed by more than one agent)

In [2]:
!python conflict_locations.py

Conflict locations written to conflict_locations.lp


Do iterative solving with malfunctions

In [28]:
!python inc_delay.py encoding_delay.lp instances/env_17_multi_agent.lp --imax 100 --delay_rate 0.2 --min_duration 2 --max_duration 5

inc-delay version 1.0
Reading from encoding_delay.lp ...
getting number of agents from instances/env_17_multi_agent.lp
there are 2 agents
2
Solving...
delay is created at step 1 for agent 0 and lasts 3
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
delay is created at step 8 for agent 1 and lasts 2
Solving...
delay is created at step 9 for agent 1 and lasts 5
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
delay is created at step 16 for agent 1 and lasts 3
Solving...
Solving...
delay is created at step 18 for agent 1 and lasts 4
Solving...
Solving...
Solving...
Solving...
Solving...
delay is created at step 23 for agent 1 and lasts 2
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
Solving...
delay is created at step 34 for agent 0 and lasts 3
Solving...
Solving...
Solving...
Answer: 1
delay(agent(0),3,1) delay(agent(0),0,2) delay(agent(1),0,3) delay(agent(0),0,4) de